<h1 align="center">Introduction to SimpleITKv4 Registration</h1>


<img src="ITKv4RegistrationComponentsDiagram.svg" style="width:700px"/><br><br>

SimpleITK caveats:
<ul>
<li>
Registraiton is either 2D/2D or 3D/3D.
</li>
<li>
Pixel types are either sitkFloat32 or sitkFloat64 (use the SimpleITK <a href="http://www.itk.org/SimpleITKDoxygen/html/namespaceitk_1_1simple.html#af8c9d7cc96a299a05890e9c3db911885">Cast()</a> function if your image's pixel type is something else).
</li>
</ul>

There are many options for creating an instance of the registration framework, all of which are configured in SimpleITK via methods of the <a href="http://www.itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1ImageRegistrationMethod.html">ImageRegistrationMethod</a> class. This class encapsulates many of the components available in ITK for constructing a registration instance.

## Registration Components 

Currently, the available choices from the following groups of ITK components are:

### Optimizers

The SimpleITK registration framework supports several optimizer types via the SetMetricAsX() methods, these include:

<ul>
  <li>
  <a href="http://www.itk.org/Doxygen/html/classitk_1_1ExhaustiveOptimizerv4.html">Exhaustive</a>
  </li>
  <li>
  <a href="http://www.itk.org/Doxygen/html/classitk_1_1AmoebaOptimizerv4.html">Nelder-Mead downhill simplex</a>, a.k.a. Amoeba.
  </li>
  <li>
  Variations on gradient descent:
  <ul>
    <li>
    <a href="http://www.itk.org/Doxygen/html/classitk_1_1GradientDescentOptimizerv4Template.html">GradientDescent</a>
    </li>
    <li>
    <a href="http://www.itk.org/Doxygen/html/classitk_1_1GradientDescentLineSearchOptimizerv4Template.html">GradientDescentLineSearch</a>
    </li>
    <li>
    <a href="http://www.itk.org/Doxygen/html/classitk_1_1RegularStepGradientDescentOptimizerv4.html">RegularStepGradientDescent</a>
    </li>
  </ul>
  </li>
  <li>
    <a href="http://www.itk.org/Doxygen/html/classitk_1_1ConjugateGradientLineSearchOptimizerv4Template.html">ConjugateGradientLineSearch</a> 
  </li>
  <li>
  <a href="http://www.itk.org/Doxygen/html/classitk_1_1LBFGSBOptimizerv4.html">L-BFGS-B</a> (Limited memory Broyden,  Fletcher,Goldfarb,Shannon-Bound Constrained) - supports the use of simple constraints ($l\leq x \leq u$)  
  </li>
</ul>

 
### Similarity metrics

The SimpleITK registration framework supports several metric types via the SetMetricAsX() methods, these include:

<ul>
  <li>
  <a href="http://www.itk.org/Doxygen/html/classitk_1_1MeanSquaresImageToImageMetricv4.html">MeanSquares</a>
  </li>
  <li>
  <a href="http://www.itk.org/Doxygen/html/classitk_1_1DemonsImageToImageMetricv4.html">Demons</a>
  </li>
  <li>
  <a href="http://www.itk.org/Doxygen/html/classitk_1_1CorrelationImageToImageMetricv4.html">Correlation</a>
  </li>
  <li>
  <a href="http://www.itk.org/Doxygen/html/classitk_1_1ANTSNeighborhoodCorrelationImageToImageMetricv4.html">ANTSNeighborhoodCorrelation</a>
  </li>
  <li>
  <a href="http://www.itk.org/Doxygen/html/classitk_1_1JointHistogramMutualInformationImageToImageMetricv4.html">JointHistogramMutualInformation</a>
  </li>
  <li>
  <a href="http://www.itk.org/Doxygen/html/classitk_1_1MattesMutualInformationImageToImageMetricv4.html">MattesMutualInformation</a>
  </li>
</ul>


### Interpolators

The SimpleITK registration framework supports several interpolators via the SetInterpolator() method, which receives one of
the <a href="http://www.itk.org/SimpleITKDoxygen/html/namespaceitk_1_1simple.html#a7cb1ef8bd02c669c02ea2f9f5aa374e5">following enumerations</a>:
<ul>
<li> sitkNearestNeighbor </li>
<li> sitkLinear </li>
<li> sitkBSpline </li>
<li> sitkGaussian </li>
<li> sitkHammingWindowedSinc </li>
<li> sitkCosineWindowedSinc </li>
<li> sitkWelchWindowedSinc </li>
<li> sitkLanczosWindowedSinc </li>
<li> sitkBlackmanWindowedSinc </li>
</ul>

In [ ]:
import SimpleITK as sitk

from __future__ import print_function

# Utility method that either downloads data from the MIDAS repository or
# if already downloaded returns the file name for reading from disk (cached data).
from downloaddata import fetch_data as fdata

# Always write output to a separate directory, we don't want to pollute the source directory. 
import os
OUTPUT_DIR = 'Output'

## Utility functions 

Callback functions for image display and for ploting the similarity metric during registration.

In [ ]:
%matplotlib inline
%run registration_utilities.py

## Read images

We first read the images, casting the pixel type to that required for registration (Float32 or Float64) and look at them.

In [ ]:
fixed_image =  sitk.ReadImage(fdata("RIRE/training_001_ct.mha"), sitk.sitkFloat32)
moving_image = sitk.ReadImage(fdata("RIRE/training_001_mr_T1.mha"), sitk.sitkFloat32) 

interact(lambda image1_z, image2_z, image1, image2,:display_scalar_images(image1_z, image2_z, image1, image2, 
                                                                          title1='fixed image',
                                                                          title2 = 'moving image'),
         image1_z=(0,fixed_image.GetSize()[2]-1), 
         image2_z=(0,moving_image.GetSize()[2]-1), 
         image1 = fixed(fixed_image), 
         image2=fixed(moving_image));

## Initial Alignment

Use the CenteredTransformInitializer to align the centers of the two volumes and set the center of rotation to the center of the fixed image.

In [ ]:
initial_transform = sitk.CenteredTransformInitializer(fixed_image, 
                                                      moving_image, 
                                                      sitk.Euler3DTransform(), 
                                                      sitk.CenteredTransformInitializerFilter.GEOMETRY)

display_registration_results(fixed_image, moving_image, initial_transform)
print(initial_transform)

## Registration - Understanding Your Input and Output

The specific registration task at hand estimates a 3D rigid transformation between images of different modalities. There are multiple components from each group (optimizers, similarity metrics, interpolators) that are appropriate for the task. Note that each component selection requires setting some parameter values. We have made the following choices:

<ul>
<li>Similarity metric, mutual information (Mattes MI):
<ul>
  <li>Number of histogram bins, 50.</li>
  <li>Sampling strategy, random.</li>
  <li>Sampling percentage, 1%.</li>
</ul>
</li>
<li>Interpolator, sitkLinear.</li>
<li>Optimizer, gradient descent: 
<ul>
  <li>Learning rate, step size along traversal direction in parameter space, 1.0 .</li>
  <li>Number of iterations, maximal number of iterations, 60.</li>
</ul>
</li>
</ul>



### Version 1 - Classic

Use the v4 registration framework in the ITK v3 manner. 

In [ ]:
registration_method = sitk.ImageRegistrationMethod()

registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
registration_method.SetMetricSamplingPercentage(0.01)

registration_method.SetInterpolator(sitk.sitkLinear)

registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=60)
registration_method.SetOptimizerScalesFromPhysicalShift() 

registration_method.SetInitialTransform(initial_transform, inPlace=False)

registration_method.AddCommand(sitk.sitkStartEvent, metric_start_plot)
registration_method.AddCommand(sitk.sitkEndEvent, metric_end_plot)
registration_method.AddCommand(sitk.sitkIterationEvent, 
                               lambda: metric_plot_values(registration_method))

final_transform_v1 = registration_method.Execute(fixed_image, moving_image)

print(final_transform_v1)

In [ ]:
display_registration_results(fixed_image, moving_image, final_transform_v1)
print('Final metric value: {0}'.format(registration_method.GetMetricValue()))
print('Optimizer\'s stopping condition, {0}'.format(registration_method.GetOptimizerStopConditionDescription()))

### Version 1.1 the v4 approach

The previous example illustrated the use of the ITK v4 registration framework in an ITK v3 manner. We only referred to a single transformation which was what we optimized.

In ITK v4 the registration method accepts three transformations (if you look at the diagram above you will only see two transformations, Moving transform represents $T_{opt} \circ T_m$):
<ul>
<li>
SetInitialTransform, $T_{opt}$ - composed with the moving initial transform, maps points from the virtual image domain to the moving image domain, modified during optimization. 
</li>
<li>
SetFixedInitialTransform $T_f$- maps points from the virtual image domain to the fixed image domain, never modified.
</li>
<li>
SetMovingInitialTransform $T_m$- maps points from the virtual image domain to the moving image domain, never modified.
</li>
</ul>

The transformation that maps points from the fixed to moving image domains is thus: $^M\mathbf{p}  = T_{opt}(T_m(T_f^{-1}(^F\mathbf{p})))$

We now modify the previous example to use $T_{opt}$ and $T_m$.

In [ ]:
registration_method = sitk.ImageRegistrationMethod()
registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
registration_method.SetMetricSamplingPercentage(0.01)
registration_method.SetInterpolator(sitk.sitkLinear)
registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=60)
registration_method.SetOptimizerScalesFromPhysicalShift()

# Set the initial moving and optimized transforms.
optimized_transform = sitk.Euler3DTransform()    
registration_method.SetMovingInitialTransform(initial_transform)
registration_method.SetInitialTransform(optimized_transform)

registration_method.AddCommand(sitk.sitkStartEvent, metric_start_plot)
registration_method.AddCommand(sitk.sitkEndEvent, metric_end_plot)
registration_method.AddCommand(sitk.sitkIterationEvent, 
                               lambda: metric_plot_values(registration_method))

registration_method.Execute(sitk.Cast(fixed_image, sitk.sitkFloat32), 
                            sitk.Cast(moving_image, sitk.sitkFloat32))

# Need to compose the transformations after registration.
final_transform_v11 = sitk.Transform(optimized_transform)
final_transform_v11.AddTransform(initial_transform)

print(final_transform_v11)

In [ ]:
display_registration_results(fixed_image, moving_image, final_transform_v11)
print('Final metric value: {0}'.format(registration_method.GetMetricValue()))
print('Optimizer\'s stopping condition, {0}'.format(registration_method.GetOptimizerStopConditionDescription()))

## Saving your results

In the end, if we are satisfied with the results we would like to save them. This is all but trivial:

In [ ]:
# Resample the moving image onto the fixed image's grid.
moving_resampled = sitk.Resample(moving_image, fixed_image, final_transform_v11, sitk.sitkLinear, 
                                 0.0, moving_image.GetPixelIDValue())    
sitk.WriteImage(moving_resampled, os.path.join(OUTPUT_DIR, 'moving_transformed.mha'))
sitk.WriteTransform(final_transform_v11, os.path.join(OUTPUT_DIR, 'final_transform_v1.1.tfm'))

## Going multi-resolution and using reference data

ITKv4 introduced an easy to use multi-resolution framework which we will explore in the following example. We  
also incorporate the usage of reference data to evaluate the quality of registration. 

What does the usage of reference data have to do with SimpleITK registration?
Short answer - Nothing.
Long answer - registration in general is dependent on many parameter settings. You will want to optimize these settings for your particular task. This requires that you evaluate results using reference data. Often this reference data comes in the form of corresponding points in the two images, and our quantitative evaluation of registration corresponds to the Target Registration Error (TRE). 

In this notebook we are using the data from the Retrospective Image Registration Evaluation (<a href=\"http://www.insight-journal.org/rire/\">RIRE</a>) project.

### Load our reference data

In [ ]:
fixed_fiducial_points, moving_fiducial_points = load_RIRE_ground_truth(fdata("RIRE/ct_T1.standard"))

# Estimate the reference_transform defined by the RIRE fiducials  
R, t = absolute_orientation_m(fixed_fiducial_points, moving_fiducial_points)
reference_transform = sitk.Euler3DTransform()
reference_transform.SetMatrix(R.flatten())
reference_transform.SetTranslation(t)

# Generate a reference dataset from the reference transformation 
# (corresponding points in the fixed and moving images).
fixed_points = generate_random_pointset(image=fixed_image, num_points=100)
moving_points = [reference_transform.TransformPoint(p) for p in fixed_points]    

# Compute the TRE prior to registration.
pre_errors_mean, pre_errors_std, _, pre_errors_max, pre_errors = registration_errors(sitk.Euler3DTransform(), fixed_points, moving_points)
print('Before registration, errors (TRE) in millimeters, mean(std): {:.2f}({:.2f}), max: {:.2f}'.format(pre_errors_mean, pre_errors_std, pre_errors_max))

### Evaluate the registration after initial alignment

In [ ]:
# The initial transform was estimated at the begining of this notebook.
initial_errors_mean, initial_errors_std, _, initial_errors_max, initial_errors = registration_errors(initial_transform, fixed_points, moving_points, display_errors=True)
print('After initial alignment, errors (TRE) in millimeters, mean(std): {:.2f}({:.2f}), max: {:.2f}'.format(initial_errors_mean, initial_errors_std, initial_errors_max))

### Version 2 - v4 multi-resolution approach 

In [ ]:
registration_method = sitk.ImageRegistrationMethod()

registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
registration_method.SetMetricSamplingPercentage(0.01)
registration_method.SetInterpolator(sitk.sitkLinear)
registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=60) 
registration_method.SetOptimizerScalesFromPhysicalShift() 

# Make a copy of the initial transformation so that we optimize in place but can still reuse it. This is
# specific to our notebook's workflow. In a regular regisration setting we would not create this copy. Also declare
# the variable as global so that it is accessible outside this cell even when using timeit.
global final_transform_multi_res
final_transform_multi_res = sitk.Euler3DTransform(initial_transform)
registration_method.SetInitialTransform(final_transform_multi_res)
registration_method.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()

# Multi-resolution framework - it's this easy:
registration_method.SetShrinkFactorsPerLevel(shrinkFactors = [4,2,1])
registration_method.SetSmoothingSigmasPerLevel(smoothingSigmas = [2,1,0])

registration_method.AddCommand(sitk.sitkStartEvent, metric_and_reference_start_plot)
registration_method.AddCommand(sitk.sitkEndEvent, metric_and_reference_end_plot)
registration_method.AddCommand(sitk.sitkIterationEvent, 
                               lambda: metric_and_reference_plot_values(registration_method, fixed_points, moving_points))
# Additional callback specific to the multi-resolution framework
registration_method.AddCommand(sitk.sitkMultiResolutionIterationEvent, 
                               metric_update_multires_iterations) 

registration_method.Execute(sitk.Cast(fixed_image, sitk.sitkFloat32), 
                            sitk.Cast(moving_image, sitk.sitkFloat32))
print('Final metric value: {0}'.format(registration_method.GetMetricValue()))
print('Optimizer\'s stopping condition, {0}\n'.format(registration_method.GetOptimizerStopConditionDescription()))
print(final_transform_multi_res)

### Evaluate the registration after final alignment

Qualitative evaluation via visual inspection and quantitative evaluation via TRE. 

In [ ]:
display_registration_results(fixed_image, moving_image, final_transform_multi_res)

In [ ]:
final_errors_mean, final_errors_std, _, final_errors_max, final_errors = registration_errors(final_transform_multi_res, fixed_points, moving_points, display_errors=True)
print('After final alignment, errors (TRE) in millimeters, mean(std): {:.2f}({:.2f}), max: {:.2f}'.format(final_errors_mean, final_errors_std, final_errors_max))

## Exercises

In this section you will explore the effects of various settings on the registration framework:
<ol>
<li>
Modify the initial alignment estimation to use sitk.CenteredTransformInitializerFilter.MOMENTS - does this have an effect on our registration?
</li>
<li>
Modify version 1 above so that the transformation is modified in place (SetInitialTransform) - does this have any effect on the result (hint: transformation type)
</li>
<li>
Modify version 1 or 1.1, exploring the effects of various component settings on the results:
<ol>
<li>
  Scale estimation: Comment out the SetOptimizerScalesFromPhysicalShift method, try the SetOptimizerScalesFromIndexShift,SetOptimizerScalesFromJacobian and SetOptimizerScales methods. 
<y t/li>
<li>
Modify the number of iterations - we already know the optimizer is likely terminating earlier than it should.
</li>
<li>
Modify the sampling strategy (SetMetricSamplingStrategy) - try no/regular sampling (registration_method.NONE, REGULAR).
</li>
</ol>
</li>
<li>
Modify version 2, exploring the effects of changing the smoothingSigmas and shrinkFactors on accuracy. You can also modify the units of the smoothingSigmas from the default physical ones to voxel units (SmoothingSigmasAreSpecifiedInPhysicalUnitsOff).
</li>
<li>
Compare the runtime of the single resolution registration, version 1, to that of the multi-resolution registration, version 2. Add the following line at the begining of the registration cells:<br>%%timeit -r1 -n1<br>
This will wrap the cell as a function and run it once, reporting the runtime. 
</li>
<li>
In registration version 2 the number of samples per resolution was set at 1%, SetMetricSamplingPercentage(0.01), this ignores the fact that there are far fewer voxels at the higher resolutions. Use the method SetMetricSamplingPercentagePerLevel with a list of percentages corresponding to the number of resolutions.  
</li>
</ol>